In [4]:
import getpass
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
load_dotenv()
from langchain_pinecone import PineconeVectorStore
from pprint import pprint
from langchain.prompts import ChatPromptTemplate




In [8]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["PINECONE_API_KEY"] = os.getenv('PINECONE_API_KEY')

llm = ChatOpenAI(model="gpt-4o")

index_name = "langchain-index"
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# Make actual retriever 
vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
    )

In [11]:
template = """Answer the question using the context. Give a one line summary of the context metadata used at the end of your answer:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

In [12]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

pprint(rag_chain.invoke("What are Biden's tax policies for young adults"))

("President Biden's tax policies focus on extending middle-class tax cuts and "
 'ensuring no tax increases for individuals making less than $400,000 per '
 'year, while implementing additional reforms to ensure the wealthy and '
 'corporations pay their fair share to avoid adding to the debt.\n'
 '\n'
 "Context Summary: The documents discuss President Biden's budget and tax "
 'policies aimed at extending middle-class tax cuts and opposing tax benefits '
 'for the wealthy.')
